In [1]:
import pandas as pd
import numpy as np

In [3]:
pd = pd.read_csv("datasets/EURUSD/1m.csv")

In [2]:
# Helper functions
def pivot_high(volume, length):
    return (volume == volume.rolling(window=2*length+1, center=True).max()).fillna(False)

def pivot_low(volume, length):
    return (volume == volume.rolling(window=2*length+1, center=True).min()).fillna(False)

def get_coordinates(df, condition, top_col, btm_col, ob_val_col, length):
    ob_top, ob_btm, ob_avg, ob_left = [], [], [], []
    
    for idx in range(len(df)):
        if condition[idx]:
            top = df.iloc[idx][top_col]
            btm = df.iloc[idx][btm_col]
            avg = (top + btm) / 2

            ob_top.insert(0, top)
            ob_btm.insert(0, btm)
            ob_avg.insert(0, avg)
            ob_left.insert(0, df.iloc[idx].name - pd.Timedelta(minutes=length * 15))

    return ob_top, ob_btm, ob_avg, ob_left

def remove_mitigated(ob_top, ob_btm, ob_left, ob_avg, target, bull):
    mitigated = False
    
    for idx in range(len(ob_top)):
        if (bull and target < ob_top[idx]) or (not bull and target > ob_btm[idx]):
            mitigated = True
            del ob_top[idx]
            del ob_btm[idx]
            del ob_avg[idx]
            del ob_left[idx]
            break

    return mitigated

# Main function
def order_block_detector(df, length=5, bull_ext_last=3, bear_ext_last=3, mitigation='Wick'):
    df['upper'] = df['high'].rolling(window=length).max()
    df['lower'] = df['low'].rolling(window=length).min()

    if mitigation == 'Close':
        df['target_bull'] = df['close'].rolling(window=length).min()
        df['target_bear'] = df['close'].rolling(window=length).max()
    else:
        df['target_bull'] = df['lower']
        df['target_bear'] = df['upper']

    df['os'] = np.where(df['high'].shift(length) > df['upper'], 0, 
                        np.where(df['low'].shift(length) < df['lower'], 1, np.nan)).ffill()

    df['phv'] = pivot_high(df['volume'], length)

    bull_top, bull_btm, bull_avg, bull_left = get_coordinates(
        df, df['phv'] & (df['os'] == 1), 'high', 'low', 'low', length
    )

    bear_top, bear_btm, bear_avg, bear_left = get_coordinates(
        df, df['phv'] & (df['os'] == 0), 'high', 'low', 'high', length
    )

    mitigated_bull = remove_mitigated(bull_top, bull_btm, bull_left, bull_avg, df['target_bull'].iloc[-1], True)
    mitigated_bear = remove_mitigated(bear_top, bear_btm, bear_left, bear_avg, df['target_bear'].iloc[-1], False)

    return {
        "bullish_order_blocks": {
            "top": bull_top,
            "bottom": bull_btm,
            "average": bull_avg,
            "left": bull_left
        },
        "bearish_order_blocks": {
            "top": bear_top,
            "bottom": bear_btm,
            "average": bear_avg,
            "left": bear_left
        },
        "mitigated_bull": mitigated_bull,
        "mitigated_bear": mitigated_bear
    }
